In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
import optuna
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import log_loss

In [ ]:
#import in the data 
train_df = pd.read_csv('../input/tabular-playground-series-may-2021/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-may-2021/test.csv')
test= test_df.drop("id",axis=1)

In [ ]:
#prepare the data for modeling 
label_dict = {"Class_1":0,"Class_2":1,"Class_3":2,"Class_4":3}

X = train_df.drop(['id', 'target'], axis=1)
y = train_df['target'].apply(lambda x : label_dict[x])

In [ ]:
losses = []
sk_fold = StratifiedKFold(5)
clf = CatBoostClassifier(verbose=False)
for train_idx,val_idx in sk_fold.split(X,y):
        x_train,x_valid = X.iloc[train_idx],X.iloc[val_idx]
        y_train,y_valid = y.iloc[train_idx],y.iloc[val_idx]
        clf.fit(x_train,y_train,eval_set=[(x_valid, y_valid)],early_stopping_rounds=100)
        preds = clf.predict_proba(x_valid)
        loss = log_loss(y_valid,preds)
        losses.append(loss)
        print(loss)
 

In [ ]:
print(np.average(losses))

In [ ]:
clf.fit(X,y,early_stopping_rounds=100)
preds = clf.predict_proba(test)

In [ ]:
submission_df = pd.DataFrame(preds,columns=['Class_1', 'Class_2', 'Class_3','Class_4'])
submission_df.insert(0, 'id', test_df['id'])

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('submission.csv', index=False)